In [227]:
import pandas as pd

# Smart Water Meter Data Analysis and Weather Integration

**Objective:**  
This notebook focuses on **predicting anomalies and patterns in smart water meter consumption** by combining meter readings with local weather data. The workflow covers data cleaning, exploration, and preparation for modeling (e.g., KMeans clustering or LSTM-based time series forecasting).  

**Key Steps:**

1. **Data Loading:**  
   - Load meter consumption data (`meter_data`) and weather data from multiple municipalities.  

2. **Data Cleaning:**  
   - Handle missing values in both meter and weather datasets.  
   - Remove duplicates and invalid records.  
   - Ensure consistency in date and municipality codes for merging.

3. **Data Integration:**  
   - Merge meter and weather datasets by `FECHA` (date) and `NUM_MUN_SGAB` (municipality).  

4. **Exploratory Data Analysis (EDA):**  
   - Inspect distributions, outliers, and missing value patterns.  
   - Analyze numeric and categorical features.  
   - Count distinct meters, dates, and weather coverage.

5. **Feature Engineering:**  
   - Create additional features such as log-transformed consumption, daily averages, or weather-derived metrics.  
   - Encode categorical variables appropriately (label encoding, one-hot, or embeddings depending on model choice).  

6. **Preparation for Modeling:**  
   - Normalize or standardize numeric features where needed (especially for distance-based or neural network models).  
   - Prepare sequences for time-series models (LSTM) or feature matrices for clustering (KMeans).


### **1.1 Read the data**



In [257]:
meter_data = pd.read_parquet("data/Dades_Comptadors_anonymized.parquet")

display(meter_data.head())

,POLIZA_SUMINISTRO,FECHA,CONSUMO_REAL,SECCIO_CENSAL,US_AIGUA_GEST,NUM_MUN_SGAB,NUM_DTE_MUNI,NUM_COMPLET,DATA_INST_COMP,MARCA_COMP,CODI_MODEL,DIAM_COMP
0,VECWAVDUULZDSBOP,2021-01-01,1758,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0
1,VECWAVDUULZDSBOP,2021-01-02,1854,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0
2,VECWAVDUULZDSBOP,2021-01-03,1885,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0
3,VECWAVDUULZDSBOP,2021-01-04,5676,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0
4,VECWAVDUULZDSBOP,2021-01-05,4456,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0


### Dataset:

We see that the dataset contains 12 columns:

POLIZA_SUMINISTRO – The unique supply contract or service code identifying the customer’s water connection.

FECHA – The date of the consumption record or measurement.

CONSUMO_REAL – The actual water consumption (typically in liters or cubic meters) recorded for that date.

SECCIO_CENSAL – The census section or geographic code representing the customer’s location.

US_AIGUA_GEST – The water usage management code indicating the type of user (e.g., domestic, commercial, industrial).

NUM_MUN_SGAB – A municipal identifier code related to the local water management authority.

NUM_DTE_MUNI – The department or district number within the municipality.

NUM_COMPLET – The complete meter or installation identifier code for the connection.

DATA_INST_COMP – The date the water meter or measuring device was installed.

MARCA_COMP – The brand or manufacturer of the installed water meter.

CODI_MODEL – The model code of the water meter.

DIAM_COMP – The diameter of the water meter (in millimeters), indicating its capacity or flow range.

In [229]:
display(meter_data.info(show_counts=True))
display(meter_data.describe())
display(meter_data.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17112709 entries, 0 to 17112708
Data columns (total 12 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   POLIZA_SUMINISTRO  17112709 non-null  object 
 1   FECHA              17112709 non-null  object 
 2   CONSUMO_REAL       17112709 non-null  int64  
 3   SECCIO_CENSAL      5800000 non-null   float64
 4   US_AIGUA_GEST      5800000 non-null   object 
 5   NUM_MUN_SGAB       5800000 non-null   float64
 6   NUM_DTE_MUNI       5800000 non-null   float64
 7   NUM_COMPLET        5800000 non-null   object 
 8   DATA_INST_COMP     5800000 non-null   object 
 9   MARCA_COMP         5800000 non-null   object 
 10  CODI_MODEL         5800000 non-null   float64
 11  DIAM_COMP          5800000 non-null   float64
dtypes: float64(5), int64(1), object(6)
memory usage: 1.5+ GB


None

,CONSUMO_REAL,SECCIO_CENSAL,NUM_MUN_SGAB,NUM_DTE_MUNI,CODI_MODEL,DIAM_COMP
count,1.711271e+07,5.800000e+06,5.800000e+06,5.800000e+06,5.800000e+06,5.800000e+06
mean,3.234748e+02,8.087469e+08,8.442387e+00,4.018163e+00,2.801573e+01,1.503135e+01
std,1.538661e+03,5.612809e+06,7.516852e+00,1.953643e+00,1.101337e+01,6.757820e-01
min,0.000000e+00,8.019010e+08,0.000000e+00,1.000000e+00,2.000000e+00,1.500000e+01
25%,9.000000e+01,8.019071e+08,0.000000e+00,3.000000e+00,3.100000e+01,1.500000e+01
50%,1.940000e+02,8.101030e+08,1.000000e+01,4.000000e+00,3.100000e+01,1.500000e+01
75%,3.290000e+02,8.101060e+08,1.000000e+01,6.000000e+00,3.100000e+01,1.500000e+01
max,5.936400e+05,8.301020e+08,4.700000e+01,1.000000e+01,7.300000e+01,3.000000e+01


POLIZA_SUMINISTRO           0
FECHA                       0
CONSUMO_REAL                0
SECCIO_CENSAL        11312709
US_AIGUA_GEST        11312709
NUM_MUN_SGAB         11312709
NUM_DTE_MUNI         11312709
NUM_COMPLET          11312709
DATA_INST_COMP       11312709
MARCA_COMP           11312709
CODI_MODEL           11312709
DIAM_COMP            11312709
dtype: int64

# Initial Observations from Dataset

## Dataset Overview
- **Number of columns:** 12  
- **Number of rows:** 17M → huge dataset

## Missing Values
- **Columns with no missing values:**  
  - `POLIZA_SUMINISTRO`   
  - `FECHA` 
  - `CONSUMO_REAL` 

- **Columns with (many) missing values:**  
  - `SECCIO_CENSAL` 
  - `US_AIGUA_GEST` 
  - `NUM_MUN_SGAB` 
  - `NUM_DTE_MUNI` 
  - `NUM_COMPLET`
  - `DATA_INST_COMP`  
  - `MARCA_COMP`  
  - `CODI_MODEL`  
  - `DIAM_COMP` 

## Data Types Observations
- **Dates:** currently `object` → should be converted to `datetime`
- **Numeric columns** (`SECCIO_CENSAL`, `NUM_MUN_SGAB`, `CODI_MODEL`, `DIAM_COMP`) are `float64` → could be optimized
- **Categorical columns** (`POLIZA_SUMINISTRO`, `US_AIGUA_GEST`, `NUM_COMPLET`, `MARCA_COMP`) → can be converted to `category` for memory savings


There are a lot of rows of missing values from attributes coming from meter data. This and the fact that all attributes with missing data had the same amount of missing values led us to think that those are the values from dates where the meter wasn't yet installed. We will filter by reading date >= installation date to see if we get rid of the missing values.

## Filter rows where reading date ≥ installation date:

In [230]:
# Only keep rows where FECHA >= DATA_INST_COMP
meter_data_filtered = meter_data[meter_data['FECHA'] >= meter_data['DATA_INST_COMP']]

print("Original dataset size:", meter_data.shape)
print("Filtered dataset size:", meter_data_filtered.shape)

# Quick peek
meter_data_filtered.head()

Original dataset size: (17112709, 12)
Filtered dataset size: (5800000, 12)


,POLIZA_SUMINISTRO,FECHA,CONSUMO_REAL,SECCIO_CENSAL,US_AIGUA_GEST,NUM_MUN_SGAB,NUM_DTE_MUNI,NUM_COMPLET,DATA_INST_COMP,MARCA_COMP,CODI_MODEL,DIAM_COMP
0,VECWAVDUULZDSBOP,2021-01-01,1758,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0
1,VECWAVDUULZDSBOP,2021-01-02,1854,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0
2,VECWAVDUULZDSBOP,2021-01-03,1885,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0
3,VECWAVDUULZDSBOP,2021-01-04,5676,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0
4,VECWAVDUULZDSBOP,2021-01-05,4456,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0


In [231]:
display(meter_data_filtered.info(show_counts=True))
display(meter_data_filtered.describe())
display(meter_data_filtered.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 5800000 entries, 0 to 5799999
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   POLIZA_SUMINISTRO  5800000 non-null  object 
 1   FECHA              5800000 non-null  object 
 2   CONSUMO_REAL       5800000 non-null  int64  
 3   SECCIO_CENSAL      5800000 non-null  float64
 4   US_AIGUA_GEST      5800000 non-null  object 
 5   NUM_MUN_SGAB       5800000 non-null  float64
 6   NUM_DTE_MUNI       5800000 non-null  float64
 7   NUM_COMPLET        5800000 non-null  object 
 8   DATA_INST_COMP     5800000 non-null  object 
 9   MARCA_COMP         5800000 non-null  object 
 10  CODI_MODEL         5800000 non-null  float64
 11  DIAM_COMP          5800000 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 575.3+ MB


None

,CONSUMO_REAL,SECCIO_CENSAL,NUM_MUN_SGAB,NUM_DTE_MUNI,CODI_MODEL,DIAM_COMP
count,5.800000e+06,5.800000e+06,5.800000e+06,5.800000e+06,5.800000e+06,5.800000e+06
mean,2.305185e+02,8.087469e+08,8.442387e+00,4.018163e+00,2.801573e+01,1.503135e+01
std,2.778459e+02,5.612809e+06,7.516852e+00,1.953643e+00,1.101337e+01,6.757820e-01
min,0.000000e+00,8.019010e+08,0.000000e+00,1.000000e+00,2.000000e+00,1.500000e+01
25%,9.600000e+01,8.019071e+08,0.000000e+00,3.000000e+00,3.100000e+01,1.500000e+01
50%,1.910000e+02,8.101030e+08,1.000000e+01,4.000000e+00,3.100000e+01,1.500000e+01
75%,3.120000e+02,8.101060e+08,1.000000e+01,6.000000e+00,3.100000e+01,1.500000e+01
max,8.213500e+04,8.301020e+08,4.700000e+01,1.000000e+01,7.300000e+01,3.000000e+01


POLIZA_SUMINISTRO    0
FECHA                0
CONSUMO_REAL         0
SECCIO_CENSAL        0
US_AIGUA_GEST        0
NUM_MUN_SGAB         0
NUM_DTE_MUNI         0
NUM_COMPLET          0
DATA_INST_COMP       0
MARCA_COMP           0
CODI_MODEL           0
DIAM_COMP            0
dtype: int64

## Convert dates into datetime values

Missing values removed. Now we will convert dates into datetime values

In [252]:
# convert date columns to datetime
# Use errors='coerce' for installation date in case of bad formatting → converts invalid strings to NaT.
meter_data_filtered['FECHA'] = pd.to_datetime(meter_data_filtered['FECHA'])
meter_data_filtered['DATA_INST_COMP'] = pd.to_datetime(meter_data_filtered['DATA_INST_COMP'], errors='coerce')


## Check for duplicates

Typically, each meter (POLIZA_SUMINISTRO) should have one record per date. We'll check for any duplicate


In [233]:
# Check for duplicates

duplicates = meter_data_filtered.duplicated(subset=['POLIZA_SUMINISTRO', 'FECHA'])
print("Number of duplicate meter-date rows:", duplicates.sum())
display(meter_data_filtered[duplicates])

Number of duplicate meter-date rows: 81


,POLIZA_SUMINISTRO,FECHA,CONSUMO_REAL,SECCIO_CENSAL,US_AIGUA_GEST,NUM_MUN_SGAB,NUM_DTE_MUNI,NUM_COMPLET,DATA_INST_COMP,MARCA_COMP,CODI_MODEL,DIAM_COMP
2751,DA43Z5V3WLWLGK5G,2024-07-26,1796,801903025.0,C,0.0,3.0,XI33KBHIV6O2U6OO,2016-07-04,5557SZ47QZAZ56EQ,23.0,30.0
238285,T52QGCUM4PKTQ5G7,2024-07-26,192,810106011.0,D,10.0,6.0,OUH6WKSYZLMG3XKW,2017-04-19,R7GO7PZAU5F6DHFH,2.0,15.0
251376,C3PU2YHFBQRNFCMW,2024-07-26,195,810106011.0,D,10.0,6.0,6RSNISOIBM3OLYCW,2017-04-28,5557SZ47QZAZ56EQ,31.0,15.0
258648,5HYIKB7XF2HCMGEV,2024-07-26,181,810106011.0,D,10.0,6.0,WLJ5DWKE4LJGNGO4,2017-04-19,R7GO7PZAU5F6DHFH,2.0,15.0
357536,4ZCEXF4SK7O7Y2IK,2024-07-26,475,810106018.0,C,10.0,6.0,QRDXIACXBMVMVKKN,2015-10-01,5557SZ47QZAZ56EQ,31.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5208775,6EZ2B4XNNCTC6HPF,2024-07-26,74,801902110.0,D,0.0,2.0,SWQ4QOHKZUX2CQFD,2015-12-17,5557SZ47QZAZ56EQ,31.0,15.0
5213137,EQCX5D5QE7A5KQFX,2024-07-26,267,801902110.0,D,0.0,2.0,2N3TK3OF4BSQ27EI,2015-12-17,5557SZ47QZAZ56EQ,31.0,15.0
5252389,FCV2DWAQVM4RUC76,2024-07-26,235,801903059.0,D,0.0,3.0,GXBJJCWIW2HRZ6FZ,2016-05-23,R7GO7PZAU5F6DHFH,2.0,15.0
5746618,TWF5OAMIVJZ4CA7U,2024-07-26,427,801902171.0,D,0.0,2.0,Z4FQJDHXYNGHBDFF,2015-06-02,5557SZ47QZAZ56EQ,31.0,15.0


Check if duplicated rows differ in the values.

In [234]:
# Keep all duplicate rows for analysis
duplicates = meter_data_filtered[meter_data_filtered.duplicated(subset=['POLIZA_SUMINISTRO', 'FECHA'], keep=False)]

duplicates = duplicates.sort_values(by=['POLIZA_SUMINISTRO', 'FECHA']).reset_index(drop=True)
# Initialize a dictionary to store counts
diff_counts = {col: 0 for col in duplicates.columns}

# Group by meter and date
grouped = duplicates.groupby(['POLIZA_SUMINISTRO', 'FECHA'])

for _, group in grouped:
    if len(group) > 1:
        # Compare row 1 with row 0, row 2 with row 1, etc.
        for i in range(1, len(group)):
            diff = group.iloc[i] != group.iloc[i-1]
            # Increment count if there is any difference in this column
            for col, is_diff in diff.items():
                if is_diff:
                    diff_counts[col] += 1
# Remove index or auxiliary columns if needed
diff_counts_clean = {k: v for k, v in diff_counts.items() if k not in ['missing_any']}  # optional

print("Number of differing duplicate rows per column:")
for col, count in diff_counts_clean.items():
    print(f"{col}: {count}")



Number of differing duplicate rows per column:
POLIZA_SUMINISTRO: 0
FECHA: 0
CONSUMO_REAL: 0
SECCIO_CENSAL: 0
US_AIGUA_GEST: 0
NUM_MUN_SGAB: 0
NUM_DTE_MUNI: 0
NUM_COMPLET: 0
DATA_INST_COMP: 0
MARCA_COMP: 0
CODI_MODEL: 0
DIAM_COMP: 0


As they don't differ, we can drop one of the duplicated rows indistinguishably.

In [235]:
meter_data_filtered = meter_data_filtered.drop_duplicates(subset=['POLIZA_SUMINISTRO', 'FECHA'], keep='first')
duplicates_after = meter_data_filtered.duplicated(subset=['POLIZA_SUMINISTRO', 'FECHA']).sum()
print("Number of duplicates remaining:", duplicates_after)

Number of duplicates remaining: 0


In [236]:
print(meter_data_filtered[meter_data_filtered['CONSUMO_REAL'] < 0].sum)

<bound method DataFrame.sum of Empty DataFrame
Columns: [POLIZA_SUMINISTRO, FECHA, CONSUMO_REAL, SECCIO_CENSAL, US_AIGUA_GEST, NUM_MUN_SGAB, NUM_DTE_MUNI, NUM_COMPLET, DATA_INST_COMP, MARCA_COMP, CODI_MODEL, DIAM_COMP]
Index: []>


There are no rows with negative consumption

## Categorical variables analysis

In [258]:
numeric_cols = meter_data_filtered.select_dtypes(include=['int64', 'float64']).columns
print("Numeric columns:", numeric_cols.tolist())
unique_counts = meter_data_filtered.nunique()
print(unique_counts)


Numeric columns: ['CONSUMO_REAL', 'SECCIO_CENSAL', 'NUM_MUN_SGAB', 'NUM_DTE_MUNI', 'CODI_MODEL', 'DIAM_COMP']
POLIZA_SUMINISTRO    3999
FECHA                1457
CONSUMO_REAL         5430
SECCIO_CENSAL         448
US_AIGUA_GEST           3
NUM_MUN_SGAB            4
NUM_DTE_MUNI            8
NUM_COMPLET          3999
DATA_INST_COMP        453
MARCA_COMP              4
CODI_MODEL             11
DIAM_COMP               3
dtype: int64


For simplicity, we decided not to use NUM_DTE_MUNI and use NUM_MUN_SGAB as the only geographical atribute.

Transform categorical variables with get_dummies:
- Expand each categorical column into n−1 dummy variables.

- Keep interpretability.

In [260]:
pd.get_dummies(meter_data_filtered, columns=['US_AIGUA_GEST', 'CODI_MODEL', 'MARCA_COMP', 'NUM_MUN_SGAB', 'DIAM_COMP'], drop_first=True)

,POLIZA_SUMINISTRO,FECHA,CONSUMO_REAL,SECCIO_CENSAL,NUM_DTE_MUNI,NUM_COMPLET,DATA_INST_COMP,US_AIGUA_GEST_C,US_AIGUA_GEST_D,CODI_MODEL_4.0,...,CODI_MODEL_66.0,CODI_MODEL_73.0,MARCA_COMP_7WZSCJKXGT7J2Q5E,MARCA_COMP_OKV4SQYX72EBODPA,MARCA_COMP_R7GO7PZAU5F6DHFH,NUM_MUN_SGAB_10.0,NUM_MUN_SGAB_25.0,NUM_MUN_SGAB_47.0,DIAM_COMP_20.0,DIAM_COMP_30.0
0,VECWAVDUULZDSBOP,2021-01-01,1758,801903025.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,True,False,False,...,False,False,False,False,False,False,False,False,False,True
1,VECWAVDUULZDSBOP,2021-01-02,1854,801903025.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,True,False,False,...,False,False,False,False,False,False,False,False,False,True
2,VECWAVDUULZDSBOP,2021-01-03,1885,801903025.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,True,False,False,...,False,False,False,False,False,False,False,False,False,True
3,VECWAVDUULZDSBOP,2021-01-04,5676,801903025.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,True,False,False,...,False,False,False,False,False,False,False,False,False,True
4,VECWAVDUULZDSBOP,2021-01-05,4456,801903025.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,True,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799995,DQEKK3RHTFRGIGCN,2023-05-20,249,801903020.0,3.0,GW7U67SVTZOTWXV6,2015-09-15,False,True,False,...,False,False,False,False,False,False,False,False,False,False
5799996,DQEKK3RHTFRGIGCN,2023-05-21,692,801903020.0,3.0,GW7U67SVTZOTWXV6,2015-09-15,False,True,False,...,False,False,False,False,False,False,False,False,False,False
5799997,DQEKK3RHTFRGIGCN,2023-05-22,267,801903020.0,3.0,GW7U67SVTZOTWXV6,2015-09-15,False,True,False,...,False,False,False,False,False,False,False,False,False,False
5799998,DQEKK3RHTFRGIGCN,2023-05-23,498,801903020.0,3.0,GW7U67SVTZOTWXV6,2015-09-15,False,True,False,...,False,False,False,False,False,False,False,False,False,False


## Adding more data

We wanted to add weather details, temperature and rain, to see how it correlates with consumption. For that we loaded 4 datasets, each one containing the weather data of different cities. We want that, for each water meter record, attach the weather data for its municipality (NUM_MUN_SGAB) and date (FECHA).

In [256]:
import pandas as pd

# Load and parse date column
bcn_weather = pd.read_csv("data/bcn_weather.csv", parse_dates=["date"])
grn_weather = pd.read_csv("data/grn_weather.csv", parse_dates=["date"])
lle_weather = pd.read_csv("data/lle_weather.csv", parse_dates=["date"])
tgn_weather = pd.read_csv("data/tgn_weather.csv", parse_dates=["date"])

Reneme date to match `FECHA`

In [240]:
for df in [bcn_weather, grn_weather, lle_weather, tgn_weather]:
    df.rename(columns={"date": "FECHA"}, inplace=True)

Add a column indicating the municipality code

In [241]:
bcn_weather["NUM_MUN_SGAB"] = 0
grn_weather["NUM_MUN_SGAB"] = 10
lle_weather["NUM_MUN_SGAB"] = 25
tgn_weather["NUM_MUN_SGAB"] = 47

Concatenate weather data with relevant columns:

In [242]:
weather_cols = ["FECHA", "tavg", "tmin", "tmax", "prcp", "NUM_MUN_SGAB"]

bcn_weather = bcn_weather[weather_cols]
grn_weather = grn_weather[weather_cols]
lle_weather = lle_weather[weather_cols]
tgn_weather = tgn_weather[weather_cols]

weather_all = pd.concat([bcn_weather, grn_weather, lle_weather, tgn_weather], ignore_index=True)

In [243]:
display(bcn_weather.head())
display(grn_weather.head()) 
display(lle_weather.head())
display(tgn_weather.head())

,FECHA,tavg,tmin,tmax,prcp,NUM_MUN_SGAB
0,2021-01-01,7.2,5.3,8.5,10.1,0
1,2021-01-02,5.7,2.7,9.7,0.0,0
2,2021-01-03,5.4,1.4,9.7,0.0,0
3,2021-01-04,4.9,0.5,11.3,0.0,0
4,2021-01-05,3.8,0.6,10.1,0.0,0


,FECHA,tavg,tmin,tmax,prcp,NUM_MUN_SGAB
0,2021-01-01,4.9,0.2,9.8,0.0,10
1,2021-01-02,1.9,-2.0,7.3,0.0,10
2,2021-01-03,2.3,-3.7,9.1,0.0,10
3,2021-01-04,0.9,-4.9,10.0,0.0,10
4,2021-01-05,2.5,-1.7,9.2,0.0,10


,FECHA,tavg,tmin,tmax,prcp,NUM_MUN_SGAB
0,2021-01-01,5.5,1.9,9.7,0.0,25
1,2021-01-02,3.2,-1.8,9.1,0.0,25
2,2021-01-03,3.4,-2.3,9.8,0.0,25
3,2021-01-04,2.4,-2.0,8.2,0.0,25
4,2021-01-05,0.0,-3.1,4.3,0.0,25


,FECHA,tavg,tmin,tmax,prcp,NUM_MUN_SGAB
0,2021-01-01,7.1,2.5,11.6,0.4,47
1,2021-01-02,6.9,3.0,10.7,0.0,47
2,2021-01-03,6.5,1.9,11.2,0.0,47
3,2021-01-04,5.5,-1.0,10.8,0.0,47
4,2021-01-05,3.1,-2.2,10.3,0.0,47


In [244]:
weather_all['FECHA'] = pd.to_datetime(weather_all['FECHA'])
display(weather_all.info(show_counts=True))
display(weather_all.describe())
display(weather_all.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5844 entries, 0 to 5843
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   FECHA         5844 non-null   datetime64[ns]
 1   tavg          5844 non-null   float64       
 2   tmin          5844 non-null   float64       
 3   tmax          5844 non-null   float64       
 4   prcp          5740 non-null   float64       
 5   NUM_MUN_SGAB  5844 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 274.1 KB


None

,FECHA,tavg,tmin,tmax,prcp,NUM_MUN_SGAB
count,5844,5844.000000,5844.000000,5844.000000,5740.000000,5844.000000
mean,2023-01-01 00:00:00.000000256,16.555955,11.441667,22.709685,1.215993,20.500000
min,2021-01-01 00:00:00,-2.200000,-6.700000,0.600000,0.000000,0.000000
25%,2022-01-01 00:00:00,11.200000,6.100000,16.800000,0.000000,7.500000
50%,2023-01-01 00:00:00,16.300000,11.400000,22.400000,0.000000,17.500000
75%,2024-01-01 00:00:00,22.400000,17.300000,28.525000,0.000000,30.500000
max,2024-12-31 00:00:00,32.500000,26.800000,43.200000,144.200000,47.000000
std,NaN,6.841088,7.031227,7.327071,5.321362,17.700384


FECHA             0
tavg              0
tmin              0
tmax              0
prcp            104
NUM_MUN_SGAB      0
dtype: int64

In [245]:
min_date = weather_all['FECHA'].min()
max_date = weather_all['FECHA'].max()

print("Date range:")
print("From:", min_date)
print("To:", max_date)


Date range:
From: 2021-01-01 00:00:00
To: 2024-12-31 00:00:00


In [247]:
weather_all[weather_all['prcp'].isna()]['NUM_MUN_SGAB'].value_counts()


NUM_MUN_SGAB
25    45
0     28
10    28
47     3
Name: count, dtype: int64

We set precipitation missing values as 0:

In [248]:
weather_all['prcp'].fillna(0, inplace=True)

C:\Users\jordi\AppData\Local\Temp\ipykernel_21032\3371632934.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weather_all['prcp'].fillna(0, inplace=True)


In [249]:
weather_all[weather_all['prcp'].isna()]['NUM_MUN_SGAB'].value_counts()


Series([], Name: count, dtype: int64)

In [250]:
meter_weather = meter_data_filtered.merge(
    weather_all,
    on=["FECHA", "NUM_MUN_SGAB"],
    how="left"
)

In [251]:
display(meter_weather.head())
display(meter_weather.info(show_counts=True))
display(meter_weather.describe())
display(meter_weather.isna().sum())

,POLIZA_SUMINISTRO,FECHA,CONSUMO_REAL,SECCIO_CENSAL,US_AIGUA_GEST,NUM_MUN_SGAB,NUM_DTE_MUNI,NUM_COMPLET,DATA_INST_COMP,MARCA_COMP,CODI_MODEL,DIAM_COMP,tavg,tmin,tmax,prcp
0,VECWAVDUULZDSBOP,2021-01-01,1758,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0,7.2,5.3,8.5,10.1
1,VECWAVDUULZDSBOP,2021-01-02,1854,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0,5.7,2.7,9.7,0.0
2,VECWAVDUULZDSBOP,2021-01-03,1885,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0,5.4,1.4,9.7,0.0
3,VECWAVDUULZDSBOP,2021-01-04,5676,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0,4.9,0.5,11.3,0.0
4,VECWAVDUULZDSBOP,2021-01-05,4456,801903025.0,C,0.0,3.0,N5ER4KUNPNXOQQCE,2016-04-25,5557SZ47QZAZ56EQ,23.0,30.0,3.8,0.6,10.1,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5799919 entries, 0 to 5799918
Data columns (total 16 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   POLIZA_SUMINISTRO  5799919 non-null  object        
 1   FECHA              5799919 non-null  datetime64[ns]
 2   CONSUMO_REAL       5799919 non-null  int64         
 3   SECCIO_CENSAL      5799919 non-null  float64       
 4   US_AIGUA_GEST      5799919 non-null  object        
 5   NUM_MUN_SGAB       5799919 non-null  float64       
 6   NUM_DTE_MUNI       5799919 non-null  float64       
 7   NUM_COMPLET        5799919 non-null  object        
 8   DATA_INST_COMP     5799919 non-null  datetime64[ns]
 9   MARCA_COMP         5799919 non-null  object        
 10  CODI_MODEL         5799919 non-null  float64       
 11  DIAM_COMP          5799919 non-null  float64       
 12  tavg               5799919 non-null  float64       
 13  tmin               5799919 

None

,FECHA,CONSUMO_REAL,SECCIO_CENSAL,NUM_MUN_SGAB,NUM_DTE_MUNI,DATA_INST_COMP,CODI_MODEL,DIAM_COMP,tavg,tmin,tmax,prcp
count,5799919,5.799919e+06,5.799919e+06,5.799919e+06,5.799919e+06,5799919,5.799919e+06,5.799919e+06,5.799919e+06,5.799919e+06,5.799919e+06,5.799919e+06
mean,2023-01-01 18:58:11.528623616,2.305178e+02,8.087469e+08,8.442378e+00,4.018173e+00,2016-07-09 05:09:29.094087680,2.801576e+01,1.503134e+01,1.612640e+01,1.085586e+01,2.282370e+01,1.338465e+00
min,2021-01-01 00:00:00,0.000000e+00,8.019010e+08,0.000000e+00,1.000000e+00,2013-05-15 00:00:00,2.000000e+00,1.500000e+01,-2.200000e+00,-6.700000e+00,6.000000e-01,0.000000e+00
25%,2022-01-01 00:00:00,9.600000e+01,8.019071e+08,0.000000e+00,3.000000e+00,2016-03-09 00:00:00,3.100000e+01,1.500000e+01,1.080000e+01,5.400000e+00,1.690000e+01,0.000000e+00
50%,2023-01-02 00:00:00,1.910000e+02,8.101030e+08,1.000000e+01,4.000000e+00,2016-06-20 00:00:00,3.100000e+01,1.500000e+01,1.580000e+01,1.090000e+01,2.230000e+01,0.000000e+00
75%,2023-12-31 00:00:00,3.120000e+02,8.101060e+08,1.000000e+01,6.000000e+00,2017-01-27 00:00:00,3.100000e+01,1.500000e+01,2.180000e+01,1.670000e+01,2.840000e+01,1.000000e-01
max,2024-12-31 00:00:00,8.213500e+04,8.301020e+08,4.700000e+01,1.000000e+01,2020-07-20 00:00:00,7.300000e+01,3.000000e+01,3.250000e+01,2.680000e+01,4.320000e+01,1.442000e+02
std,NaN,2.778461e+02,5.612814e+06,7.516832e+00,1.953639e+00,NaN,1.101337e+01,6.757581e-01,6.566246e+00,7.097138e+00,7.028850e+00,5.308267e+00


POLIZA_SUMINISTRO    0
FECHA                0
CONSUMO_REAL         0
SECCIO_CENSAL        0
US_AIGUA_GEST        0
NUM_MUN_SGAB         0
NUM_DTE_MUNI         0
NUM_COMPLET          0
DATA_INST_COMP       0
MARCA_COMP           0
CODI_MODEL           0
DIAM_COMP            0
tavg                 0
tmin                 0
tmax                 0
prcp                 0
dtype: int64


### **1.2 Dataset Exploratory Data Analysis (EDA)**